# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---5311'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2018-07-09 to 2021-07-08
You have 239 stocks
['AAL', 'AAPL', 'ABBV', 'ACN', 'ADI', 'ADP', 'ADSK', 'AEI', 'AEP', 'AFL', 'AIG', 'AIH', 'ALL', 'ALXN', 'AMAT', 'AMP', 'AMT', 'AMZN', 'ANET', 'APTV', 'AVGO', 'AZO', 'BABA', 'BAC', 'BAX', 'BBY', 'BDX', 'BIIB', 'BK', 'BLK', 'BLL', 'BRK.B', 'BSX', 'C', 'CAN', 'CARR', 'CAT', 'CB', 'CCI', 'CCL', 'CERN', 'CFG', 'CI', 'CL', 'CLX', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COST', 'CPB', 'CSCO', 'CSX', 'CTSH', 'CVS', 'CVX', 'D', 'DFS', 'DG', 'DHI', 'DKNG', 'DLR', 'DOV', 'DPZ', 'DRI', 'DUK', 'DXCM', 'EA', 'EBAY', 'ECL', 'EFOI', 'EL', 'EMR', 'ENPH', 'EOG', 'ETSY', 'EW', 'EXPE', 'F', 'FANG', 'FCX', 'FDX', 'FISV', 'FLT', 'FRC', 'FTNT', 'FTV', 'GD', 'GE', 'GILD', 'GIS', 'GMGMF', 'GNRC', 'GOOG', 'GPN', 'GS', 'HAL', 'HBAN', 'HCA', 'HD', 'HES', 'HLT', 'HON', 'HPQ', 'HST', 'HSY', 'HUM', 'IBM', 'IFF', 'INFO', 'INTU', 'IR', 'JCI', 'JPM', 'KHC', 'KLAC', 'KMB', 'KMX', 'KO', 'KR', 'KSU', 'LB', 'LEN', 'LHX', 'LIN', 'L

ticker,AAL,AAPL,ABBV,ACN,ADI,ADP,ADSK,AEI,AEP,AFL,...,WELL,WFC,WLTW,WM,WMB,WMT,WY,WYNN,XOM,ZKIN
date,,,,,,,,,,,,,,,,,,,,,
2021-07-01 00:00:00+00:00,21.51,137.27,114.26,300.00,169.17,199.81,293.70,5.31,85.260,54.00,...,83.73,45.41,231.00,141.17,26.80,139.32,34.33,122.40,63.26,4.23
2021-07-02 00:00:00+00:00,21.48,139.96,115.17,304.84,170.67,201.48,297.74,5.63,85.200,54.15,...,84.92,45.07,231.70,141.97,26.94,140.11,34.31,121.50,63.17,3.95
2021-07-06 00:00:00+00:00,21.01,142.02,115.73,305.40,168.32,201.11,295.60,5.25,85.240,53.27,...,85.14,43.49,229.95,142.77,26.42,139.94,33.97,116.90,61.37,3.93
2021-07-07 00:00:00+00:00,20.31,144.57,116.75,309.61,165.43,203.34,296.11,4.86,85.620,53.76,...,84.87,43.40,230.77,144.04,26.28,139.71,34.84,112.91,60.41,3.62
2021-07-08 00:00:00+00:00,20.39,143.16,116.60,309.15,163.91,201.86,294.47,4.86,85.365,52.33,...,84.83,42.33,224.15,142.17,26.24,139.59,35.04,111.66,60.38,3.66


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean().rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().smoothed().rank().zscore().for_al(),
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.MarketDispersion(price_histories, 20).for_al(),
    alpha_factors.MarketDispersion(price_histories, 120).for_al(),
    alpha_factors.MarketVolatility(price_histories, 20).for_al(),
    alpha_factors.MarketVolatility(price_histories, 120).for_al()
], axis=1)
alpha_factors.FactorDateParts(all_factors)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAPL'].tail()

,,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day,average_dollar_volume_20_day,average_dollar_volume_120_day,market_dispersion20_day,market_dispersion120_day,market_volatility20_day,market_volatility120_day,is_January,is_December,weekday,quarter,year,month_start,month_end,quarter_start,quarter_end
date,ticker,,,,,,,,,,,,,,,,,,,,
2021-07-01 00:00:00+00:00,AAPL,0.173931,-1.478416,-1.507405,0.536288,0.014494,1.69583,1.710325,0.021279,0.026372,0.078246,0.116732,0,0,3,3,2021,1,0,1,0
2021-07-02 00:00:00+00:00,AAPL,0.231908,-1.579876,-1.434934,0.405840,-0.072471,1.69583,1.710325,0.021052,0.026292,0.076834,0.116720,0,0,4,3,2021,0,0,0,0
2021-07-06 00:00:00+00:00,AAPL,0.304380,-1.623359,-1.188531,0.376851,-0.159437,1.69583,1.710325,0.020649,0.026263,0.083953,0.117133,0,0,1,3,2021,0,0,0,0
2021-07-07 00:00:00+00:00,AAPL,0.275391,-1.608865,-0.913139,0.492805,-0.217414,1.69583,1.710325,0.020776,0.026255,0.085616,0.117080,0,0,2,3,2021,0,0,0,0
2021-07-08 00:00:00+00:00,AAPL,0.246403,-1.623359,-0.550782,0.521794,-0.289886,1.69583,1.710325,0.020963,0.026191,0.091710,0.118091,0,0,3,3,2021,0,0,0,0


In [6]:
all_factors.describe()

,momentum_252_day_logret,mean_reversion_5_day_logret_smoothed,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day,average_dollar_volume_20_day,average_dollar_volume_120_day,market_dispersion20_day,market_dispersion120_day,market_volatility20_day,market_volatility120_day,is_January,is_December,weekday,quarter,year,month_start,month_end,quarter_start,quarter_end
count,1.204560e+05,1.204560e+05,1.204560e+05,1.204560e+05,1.204560e+05,1.204560e+05,1.204560e+05,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000,120456.000000
mean,7.594672e-19,-3.170591e-19,1.253490e-18,3.465530e-19,1.474694e-18,1.054406e-18,1.865487e-18,0.024245,0.023293,0.171505,0.170024,0.079365,0.085317,2.013889,2.515873,2020.013889,0.041667,0.045635,0.011905,0.015873
std,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,1.000004e+00,0.008120,0.005414,0.157569,0.087952,0.270309,0.279355,1.396503,1.116150,0.705569,0.199827,0.208693,0.108458,0.124985
min,-1.724851e+00,-1.724840e+00,-1.724821e+00,-1.724819e+00,-1.724819e+00,-1.724819e+00,-1.724819e+00,0.013900,0.015865,0.034460,0.075153,0.000000,0.000000,0.000000,1.000000,2019.000000,0.000000,0.000000,0.000000,0.000000
25%,-8.696567e-01,-8.696567e-01,-8.696566e-01,-8.696567e-01,-8.696567e-01,-8.696567e-01,-8.696567e-01,0.018261,0.016741,0.083939,0.099900,0.000000,0.000000,1.000000,2.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.022822,0.024084,0.126722,0.141323,0.000000,0.000000,2.000000,3.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
75%,8.696567e-01,8.696567e-01,8.696566e-01,8.696567e-01,8.696567e-01,8.696567e-01,8.696567e-01,0.028613,0.026628,0.180683,0.213771,0.000000,0.000000,3.000000,4.000000,2021.000000,0.000000,0.000000,0.000000,0.000000
max,1.724851e+00,1.724840e+00,1.724821e+00,1.724840e+00,1.724840e+00,1.724840e+00,1.724840e+00,0.055844,0.033255,0.896364,0.365270,1.000000,1.000000,4.000000,4.000000,2021.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
all_factors.head()

momentum_252_day_logret  \
date                      ticker                            
2019-07-10 00:00:00+00:00 AAL                   -1.130567   
                          AAPL                  -0.028989   
                          ABBV                  -1.434951   
                          ACN                    0.434834   
                          ADI                    0.521800   

                                  mean_reversion_5_day_logret_smoothed  \
date                      ticker                                         
2019-07-10 00:00:00+00:00 AAL                                -0.768206   
                          AAPL                               -1.087084   
                          ABBV                                1.376973   
                          ACN                                -0.463823   
                          ADI                                -1.362479   

                                  overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2019-07-10 00:00:00+00:00 AAL                               0.594266   
                          AAPL                             -0.188426   
                          ABBV                             -0.623254   
                          ACN                              -1.608865   
                          ADI                               1.565382   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2019-07-10 00:00:00+00:00 AAL                        1.029106   
                          AAPL                       0.695734   
                          ABBV                       0.826184   
                          ACN                       -1.420457   
                          ADI                        1.116073   

                                  annualzed_volatility_120_day  \
date                      ticker                                 
2019-07-10 00:00:00+00:00 AAL                         0.391350   
                          AAPL                        1.420457   
                          ABBV                       -1.188545   
                          ACN                         0.434834   
                          ADI                         0.289889   

                                  average_dollar_volume_20_day  \
date                      ticker                                 
2019-07-10 00:00:00+00:00 AAL                        -0.347867   
                          AAPL                        1.710346   
                          ABBV                        1.521918   
                          ACN                         0.869667   
                          ADI                         0.376856   

                                  average_dollar_volume_120_day  \
date                      ticker                                  
2019-07-10 00:00:00+00:00 AAL                         -0.086967   
                          AAPL                         1.710346   
                          ABBV                         1.203040   
                          ACN                          0.565284   
                          ADI                          0.463823   

                                  market_dispersion20_day  \
date                      ticker                            
2019-07-10 00:00:00+00:00 AAL                    0.014314   
                          AAPL                   0.014314   
                          ABBV                   0.014314   
                          ACN                    0.014314   
                          ADI                    0.014314   

                                  market_dispersion120_day  \
date                      ticker                             
2019-07-10 00:00:00+00:00 AAL                      0.01616   
                          AAPL                     0.01616   
                          ABBV                     0.0